# Трансформеры
В этом домашнем задании мы рассмотим использование трансформеров в библиотеке PyTorch. Рассмотрим задачу языкового моделирования. Попробуем генерировать текст нейронной сетью. 

Ссылка на данные - https://drive.google.com/drive/folders/1x1A4ElliUGBPnHladGMwPxPuGxI8Vnpu?usp=sharing

In [ ]:
# хороший тон, импортировать все необходимые библиотеки в одной ячейке ;)

import torch
from torch import nn

import numpy as np
import time
from sklearn.model_selection import train_test_split
import math

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ijpKzdj4d1b0zYKot3_380OWtiI-2UJX' -O data1

--2021-12-20 16:38:43--  https://docs.google.com/uc?export=download&id=1ijpKzdj4d1b0zYKot3_380OWtiI-2UJX
Resolving docs.google.com (docs.google.com)... 74.125.124.139, 74.125.124.138, 74.125.124.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.124.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sgae7dtk2vfqrgfkl4tge9uj9avkv7rj/1640018325000/09709797766287458306/*/1ijpKzdj4d1b0zYKot3_380OWtiI-2UJX?e=download [following]
--2021-12-20 16:38:45--  https://doc-0s-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sgae7dtk2vfqrgfkl4tge9uj9avkv7rj/1640018325000/09709797766287458306/*/1ijpKzdj4d1b0zYKot3_380OWtiI-2UJX?e=download
Resolving doc-0s-54-docs.googleusercontent.com (doc-0s-54-docs.googleusercontent.com)... 209.85.200.132, 2607:f8b0:4001:c16::84
Connecting to doc-0s-54-docs.googleusercontent.com (doc-0s-54-d

Что такое языковое моделирование? Это предсказание вероятности следующего токена (слова или буквы) на основе предыдущих токенов. Математически это можно описать так:

$$P(x_i|x_1, x_2 , ... , x_{i-1})$$ 

Последовательность $$ x_1, x_2, ... x_{i-1} $$ называют контекстом.

## Задание 1 (0.5 балла)
Мы будем обучать языковую модель для предсказания следущей буквы. Такие языковые модели применяются в распозновании речи, так как предоставляют дополнительную информацию акустической модели при выборе следующего символа. Для начала, откройте файл с данными, посмотрите, какие символы входят в тексты, сколько их. Уберите из текста все символы переноса на новую строку и табуляцию.

In [ ]:
path = 'data1'
file = open(path, 'r')
data = file.readlines()
len(data)

700000

In [ ]:
print(set(''.join(data))) # Смотрим какие символы содержит текст. Из "шумовых" символов только перенос строки "\n" и дефис "-".
# Дефис не относится к табуляции, также как и пробел. Видимо, мы их оставляем.

{'з', 'л', 'с', 'м', 'к', 'а', 'ю', 'ъ', 'ш', 'ё', 'е', 'д', 'т', 'н', 'й', 'у', 'ы', 'ь', 'ф', 'г', 'ж', '-', ' ', 'п', 'р', 'и', 'о', 'б', 'я', 'ц', 'в', 'э', 'ч', 'х', '\n', 'щ'}


In [ ]:
data = [i.strip('\n') for i in data]

In [ ]:
print(len(set(''.join(data)))) # остаются 33 буквы алфавита, пробел, и дефис. 

35


## Задание 2 (0.5 балла)
Для обучения модели требуется сначала подготовить текст в подходящий для нейросети вид. Важно также отметить, что нужно добавить два токена start и end, которые отвечают за начало и конец текста. Используйте [ и ] для этой задачи. Также нам нужен токен pad, чтобы заполнять им текст до требуемой длинны для формирования батча.

Реализуйте метод preprocess класса Preprocessor. Он должен принимать на вход текст и длинну текста, которую мы ожидаем получить на выходе. Текст должен быть переведен в нижний регистр, в конец текста добавляется требуемое число pad токенов, далее текст векторизуется (каждому символу ставится свое число). Вернуть требуется два вектора. Полученный результат без последнего токена (на нем будем обучаться) и полученный результат без первого токена (целевые метки при обучении).

In [ ]:
class Preprocessor:
    def __init__(self):
        self.alphabet = '_добсркгаупитнезчм яфжлйвцыэь-шхющёъ]['
        self.token2ind = {}
        self.ind2token = {}
        for i in range(len(self.alphabet)):
            self.token2ind[self.alphabet[i]] = i
            self.ind2token[i] = self.alphabet[i]
        
    
    def preprocess(self, text, window_size):
        text = '[' + text + ']' # Добавляю токены начала ("[") и конца ("]") текста
        if len(text) < window_size: 
            text += '_'*(window_size - len(text)) # Заполняю недостающие символы pad токенами "_"
        vector = np.array([self.token2ind[tol] for tol in text.lower()]) # Перевожу текст в векторный вид
        self.train_vec = vector[:-1]
        self.test_vec = vector[1:]
        return self.train_vec, self.test_vec
        pass

## Задание 3 (0.5 балла)
Так как мы решили, что текст будет начинаться токеном [ и заканчиваться токеном ], данные нужно поправить. Реализуйте эту идею, добавьте данные токены в ваши тексты.

Токены начала и конца текста добавляются на этапе препроцессинга. Реализовано в задании 2.

## Задание 4 (0.5 балла)
Так как мы не располагаем большими мощностями, то давайте ограничим максимальную длинну текста. Вы можете менять этот порог и тем самым уменьшать кол-во текстов в вашей выборке и увеличивая тем самым скорость обучения. Начнем же мы с 128. 
Выберите порог и оставьте только те тексты, длина которых не превосходит данный порог.

Далее разбейте тексты на train и test, перемешайте тексты при разбиении, размер тестовой выборки должен быть 15% от общего числа текстов. 

In [ ]:
THRESHOLD = 128

data_cut = []

for text in data:
  if len(text) < THRESHOLD - 2: #Сделала -2, т.к. добавляю токены конца и начала т екста на этапе препроцессинга. 
    data_cut.append(text)


In [ ]:
len(data_cut)

683133

In [ ]:
data_train, data_test = train_test_split(data_cut, test_size=0.15, random_state=np.random.seed(), shuffle=True)

## Задание 5 (2 балла)
Напишем датасет. На вход датасету передается набор текстов, объект класса Preprocessor и размер окна, который вы выбрали в прошлом задании.
Реализуйте методы __len__ и __getitem__.

In [ ]:
class TextDataset(torch.utils.data.Dataset):
    
    def __init__(self, data, preproc, win_size = 128):
        self.x = []
        self.y = []
        for text in data:
          x, y = preproc.preprocess(text, win_size)
          self.x.append(x)
          self.y.append(y)

        pass
    
    def __len__(self):
        return len(self.x)
        
        pass
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
        pass

In [ ]:
preproc = Preprocessor()
train_dataset = TextDataset(data_train, preproc)
test_dataset = TextDataset(data_test, preproc)

## Задание 6 (2 балла)
Напишем модель. Класс для реализации positional encoding реализован за вас, он нужен, чтобы модель могла после получения эмбедингов понимать, на каком месте какой токен находится.

Заполните пропуски в классе модели. Гипперпараметры модели вам предлагается подобрать самостоятельно. Рекомендуется использовать не более 6 слоев в трансформере. В декореде испоьлзуйте две линейных слоя с функцией активации ReLU между ними.

## Задание 6_1 (0 баллов, но надо ответить!)
При обучении языковой модели на основе трансформеров мы используем маскирование символов (как мы это делаем - уже реализовано). Напишите, почему мы это делаем? Почему это так важно?

Маскирование необходимо чтобы ограничить доступ нашей модели к последующим символам последовательности. Мы хотим, чтобы модель обучалась на уже известных символах, не заглядывая в последующий текст.

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [ ]:
class LanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super(LanguageModel, self).__init__()
        d_model=16
        self.emb = nn.Embedding(vocab_size, d_model)
        self.pe = PositionalEncoding(d_model)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model, nhead=2, dropout=0.2)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer=self.transformer_encoder_layer, num_layers=2)
        self.decoder = nn.Sequential(
            nn.Linear(d_model, vocab_size),
            nn.ReLU(),
            nn.Linear(vocab_size, vocab_size)
        )
    
    def forward(self, x, src_mask):
        x = self.pe(self.emb(x)) # emb, then pe
        x = x.transpose(1, 0)
        x = self.transformer_encoder(self.transformer_encoder_layer(x), src_mask) # transformer encoder with mask
        x = self.decoder(x) # decoder
        return x.transpose(1, 0)
    
    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [ ]:
model = LanguageModel(len('_добсркгаупитнезчм яфжлйвцыэь-шхющёъ]['))

## Задание 7 (2,5 балла)
Финишная прямая. Давайте реализуем класс для обучения модели и ее валидации. Следуйте указаниям в коде и заполните недостающие фрагменты в коде.

In [ ]:
class Trainer:
    
    def __init__(self, model, train_dataset, test_dataset):
        
        self.model = model
        
        self.train_batch_size = 127
        self.test_batch_size = 127
        
        self.train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=self.train_batch_size, drop_last=True, shuffle=True)
        self.test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=self.test_batch_size, drop_last=True, shuffle=True)
        self.train_dataloader_size = len(train_dataset) // self.train_batch_size
        self.test_dataloader_size = len(test_dataset) // self.test_batch_size
        
        self.device = 'cuda:0'
        self.criterion = nn.CrossEntropyLoss(ignore_index=0)
        
        self.optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, 1.0, gamma=0.8)
        
        self.steps_to_print = 1000
        
    def train_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        scr_mask = self.model.generate_square_subsequent_mask(self.train_batch_size).to(self.device)
        
        for batch in self.train_dataloader:
            x, y = batch
            x = x.to(self.device)
            y = y.to(self.device)
            predicted = model(x, scr_mask) 
            loss = self.criterion(predicted.reshape(16129, 38), y.view(127*127).long())
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
            counted_loss += loss.item()
            step+=1
            it+=1
            
            
            if step%self.steps_to_print == 0:
                it = step
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
    
    def validate_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        scr_mask = self.model.generate_square_subsequent_mask(self.test_batch_size).to(self.device)
        
        for batch in self.test_dataloader:
            x, y = batch
            x = x.to(self.device)
            y = y.to(self.device)
            predicted = model(x, scr_mask)
            loss = self.criterion(predicted.reshape(16129, 38), y.view(127*127).long())
            counted_loss += loss.item()
            step+=1
            it+=1
            
            
            if step%(self.steps_to_print//2) == 0:
                it = step
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        self.model.to(self.device)
        for epoch in range(1, number_of_epochs+1):
            model.train()
            self.train_one_epoch(epoch)
            with torch.no_grad():
                model.eval()
                self.validate_one_epoch(epoch)
            print()
            self.scheduler.step()

Что такое ppl? Перплексия. Ее можно интерпретировать как меру "удивленности" модели нужному символу. Чем меньше данная величина, тем лучше, ведь это значит, что модель если и сделала неправильный выбор, то не сильно удивлена своей ошибке.

Проведите несколько экспериментов, посмотрите, при каких гипперпараметрах значение перплексии минимально.

## Задание 8 (0.5 балла)
Запустите обучение на нескольких эпохах. Ориентируйтесь на ваши вычислительные мощности и время работы. Вы всегда можете посчитать, сколько секунд уходит на один батч.

In [ ]:
# YOUR CODE HERE
trainer = Trainer(model, train_dataset, test_dataset)
trainer.train(2)


Train epoch 1 | Step 1000/4572 | Counted loss 2751.056639432907 | ppl 15.659169241344655 | time 114.18730092048645 | 
Train epoch 1 | Step 2000/4572 | Counted loss 2611.32036280632 | ppl 3.6901244412624488 | time 113.76406860351562 | 
Train epoch 1 | Step 3000/4572 | Counted loss 2582.0330657958984 | ppl 2.364762723542021 | time 113.68537139892578 | 
Train epoch 1 | Step 4000/4572 | Counted loss 2564.8264105319977 | ppl 1.8987705592164652 | time 113.70761322975159 | 
Validate epoch 1 | Step 500/806 | Counted loss 1256.403660774231 | ppl 12.339522489138021 | time 20.42834782600403 | 

Train epoch 2 | Step 1000/4572 | Counted loss 2544.4171607494354 | ppl 12.735802996585052 | time 113.83225846290588 | 
Train epoch 2 | Step 2000/4572 | Counted loss 2537.8405928611755 | ppl 3.5570099719363846 | time 113.6880292892456 | 
Train epoch 2 | Step 3000/4572 | Counted loss 2530.6955931186676 | ppl 2.324640021771023 | time 113.6796498298645 | 
Train epoch 2 | Step 4000/4572 | Counted loss 2524.7500

## Задание 9 (1 балл)
Итак, давайте попробуем погенерировать текст нашей сеткой. Закончите функцию по генерации текста. Попробуйте сгенерировать какой-нибудь текст. Помните, что если вы хотите генерировать текст с нуля, то вы должны передать в качестве текста только токен start.
Прекратите генерировать текст, если модель выдала токен end или длинна текста больше 150.

Столкнулась с проблемой "CUDA out of memory", пришлось ограничить число символов меньшим порогом.

In [ ]:
from torch._C import device
def generate_text(text):
    x = []
    
    for letter in text:
        x.append(preproc.token2ind[letter])
    x = torch.from_numpy(np.array(x))
    
    pred = model(x.to('cuda:0'), model.generate_square_subsequent_mask(len(text)).to('cuda:0'))
    ind = int(torch.argmax(pred[-1][-1].cpu()))

    text += preproc.ind2token[ind]
    
    if text[-1] == ']' or len(text) > 100:
        return text
    else:
        return generate_text(text)

In [ ]:
generate_text('[')

'[на по по по пото по по по по по по по пото по по по по по по пото по по по по по по по по по по по п'